<a href="https://colab.research.google.com/github/AliARajabi/deep-learning/blob/main/RNN/machine_translation(Uncompelet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

Download data from [manythings](https://www.manythings.org/anki/pes-eng.zip)
this link is forbidden from our country so we download it manually

In [ ]:
!wget http://www.manythings.org/anki/pes-eng.zip

--2021-04-16 16:58:49--  http://www.manythings.org/anki/pes-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112927 (110K) [application/zip]
Saving to: ‘pes-eng.zip’

pes-eng.zip         100%[===================>] 110.28K  --.-KB/s    in 0.1s    

2021-04-16 16:58:50 (841 KB/s) - ‘pes-eng.zip’ saved [112927/112927]



In [ ]:
! unzip /content/pes-eng.zip

Archive:  /content/pes-eng.zip
  inflating: _about.txt              
  inflating: pes.txt                 


In [ ]:
file_path='/content/pes.txt'


Converts the unicode file to ascii

In [ ]:
def Converts_unicode_to_ascii(text):
  return ''.join(c for c in unicodedata.normalize('NFD', text)if unicodedata.category(c) != 'Mn')

In [ ]:
def preprocess_sentence(w):
  w = Converts_unicode_to_ascii(w.lower().strip())

  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,؟(آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی)]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"       ¿می تونم این کتاب روِِِِِِِِِیِِِِِِِِِِِ|||| قرض بگیرم؟"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))

<start> may i borrow this book ? <end>
<start> می تونم این کتاب روی قرض بگیرم؟ <end>


In [ ]:
def Create_dataset(path,data_num):
  lines=io.open(file=path,encoding='UTF-8').read().strip().split('\n')
  words=[[preprocess_sentence(s) for s in line.split('\t')] for line in lines[:data_num]]
  return zip(*words)

In [ ]:
en, per,_ = Create_dataset(file_path, 3)
print(en)
print(per)

('<start> who ? <end>', '<start> go on . <end>', '<start> smile . <end>')
('<start> چه کسی؟ <end>', '<start> ادامه بده ( ادامه دادن ) <end>', '<start> لبخند بزن . <end>')


In [ ]:
def tokenize(inp):
  tok=tf.keras.preprocessing.text.Tokenizer(filters='')
  tok.fit_on_texts(inp)

  tensor=tok.texts_to_sequences(inp)
  tensor=tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')

  return tensor,tok

In [ ]:
def load_dataset(path,data_num):
  input_lang,output_lang,_=Create_dataset(path, data_num)

  input_lang_tok,input_tokenizer=tokenize(input_lang)
  output_lang_tok,output_tokenizer=tokenize(output_lang)
  return input_lang_tok,output_lang_tok,input_tokenizer,output_tokenizer

In [ ]:
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(file_path,
                                                                num_examples)

In [ ]:
print(type(input_tensor))
print(input_tensor.shape)
print(input_tensor[50])

<class 'numpy.ndarray'>
(2284, 37)
[  1  17  18 264   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


In [ ]:
print(type(target_tensor))
print(target_tensor.shape)
print(target_tensor[50])

<class 'numpy.ndarray'>
(2284, 34)
[  1 106   9   3   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [ ]:
input_tensor_train,input_tensor_val,target_tensor_train,target_tensor_val = train_test_split(input_tensor, target_tensor,test_size=0.2)

In [ ]:
print(f'input train size: {len(input_tensor_train)}, output train size: {len(target_tensor_train)}, input validation size: {len(input_tensor_val)},output validation size: {len(target_tensor_val)}')

input train size: 1827, output train size: 1827, input validation size: 457,output validation size: 457


In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print(f'{t} ----> {lang.index_word[t]}')

In [ ]:
print("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print()
print("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
14 ----> he
20 ----> was
54 ----> very
618 ----> patient
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
8 ----> او
42 ----> خیلی
1028 ----> صبور
21 ----> بود
3 ----> .
2 ----> <end>


Create a tf.data dataset

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 37]), TensorShape([64, 34]))

This Implementation uses Bahdanau attention for the encoder. Let's decide on notation before writing the simplified form:

FC = Fully connected (dense) layer

EO = Encoder output

H = hidden state

X = input to the decoder

And the pseudo-code:


* score = FC(tanh(FC(EO) + FC(H)))

* attention weights = softmax(score, axis = 1). Softmax by default is applied on the last axis but here we want to apply it on the 1st axis, since the shape of score is (batch_size, max_length, hidden_size). Max_length is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.

* context vector = sum(attention weights * EO, axis = 1). Same reason as above for choosing axis as 1.

* embedding output = The input to the decoder X is passed through an embedding layer.

* merged vector = concat(embedding output, context vector)
This merged vector is then given to the GRU
The shapes of all the vectors at each step have been specified in the comments in the code: